![image](https://hydro-jules.org/sites/default/files/Hydro-JULES_Logo_Positive.png)
# Hydro-JULES - Plotting observed data against modelled data

This notebook provides a very basic walk through of how to use observed data in csv format (from COSMOS-UK network) and compare it against modelled gridded soil moisture data in netCDF format (created for Hydro-JULES), in python using the Numpy, Cf-Python and Matplotlib packages.

For an introduction of the tutorial please see the video https://youtu.be/kqez7RtCKdk

The observed data used in this example is the volumentric water content (VWC) from COSMOS UK station network. VWC is the percentage water content of the soil. This is measured by the COSMOS stations (https://cosmos.ceh.ac.uk/) and is driven by the balance between precipitation and evaporation. The modelled data used in this example is soil water stress (SWS). SWS is a dimensionless variable typically used to determine how much soil moisture vegetation has access to for transpiration. It is modelled using the Artemis Model (https://github.com/cm4twc-org/cm4twccontrib-artemis), where it is calculated using the soil water content.



**Contents** (note: to run cells later in the notebook, the first few cells must be run in order to import packages and initialise variables):
- [Import packages required for this work](#import_packages)
- [Quick csv and netCDF file access to show how easy it is](#quick_file_access)
- [Reading the data into different numpy arrays](#explore_data)
- [Quick plotting of data](#quick_plot_data)
- [Manipulating the datasets](#manipulating_data)
- [Further exercises](#further_exercises)


<a id="import_packages"></a>
### Import packages required for this work

In [ ]:
# Importing cf-python and cfplot
import cf, cfplot as cfp

# Additional packages for advance plotting with cfplot
import cartopy.crs as ccrs
import matplotlib.patches as mpatches

# Importing matplotlib to show that data read through cf-python can be plotted using matplotlib
import matplotlib.pyplot as plt

#Importing numpy
import numpy as np
#import pandas as pd

# The following line allows matplotlib plots to be shown in the notebook
%matplotlib inline


<a id="quick_file_access"></a>
### Quick csv and NetCDF file access to show how easy it is

In [ ]:
# Initialise the data path and add file name
path = '/data/example-data/'
fsite = 'SITES_2016.csv'

# Read the station information csv file using numpy
my_stations = np.genfromtxt(path+fsite, delimiter=',', dtype='unicode') # Unicode reads data in string type which would need to be converted to float later on

# Printing the dataset read in as a check
print(my_stations)
# Columns show site_id, site_name, latitude and longitude and rows show each station information

In [ ]:
# Read the VWC data csv file using numpy
fcos = 'COSMOS_VWC_2016.csv'
my_vwc = np.genfromtxt(path+fcos, delimiter=',', dtype='unicode')
# Instead of this you can also use my_vwc = np.genfromtxt(path+fcos, delimiter=',', skip_header=1) to remove the header and just have floats
# Pandas is also a good python package to be able to use for csv tables. Currently it is not installed in DataLabs.

# Printing the dataset read in as a check
print(my_vwc)
print(my_vwc.shape) 
# There are 366 days information in rows with one extra row for the titles. There are 30 stations and one extra column for time information

In [ ]:
# Creating separate arrays for Station IDs, Names, latitudes and Longitudes without the headings
st_id = my_stations[1:,0]
st_name = my_stations[1:,1]
st_lat = my_stations[1:,2]
st_lon = my_stations[1:,3]

In [ ]:
# Initialise a file name along with its path
path = '/data/demo-data/out-example/'
file = 'demo-run-more-records_subsurface_run_records_daily.nc'

# We read in a netCDF file from the modelled output and select the soil water stress (SWS) variable
my_sws = cf.read(path+file).select('soil_water_stress')[0]

# To convert the SWC to similar units of VWC, we multiply SWC with 100
my_sws = my_sws*100
print(my_sws)

# Time mean for the SWS data
my_sws_mean = my_sws.collapse('time: mean').squeeze()


<a id="quick_plot_data"></a>
### Quick plotting of data

In [ ]:
#Cf-plot is the plotting software for cfpython
cfp.reset() # Just to reset any old settings for cfplot

#Plotting all the stations on a UK map
cfp.gopen()
cfp.mapset(latmin=49, latmax=60, lonmin=-11, lonmax=2, resolution='50m') #This line is for improved resolution than default of 110m
cfp.setvars(continent_thickness=0.5) #continent_thickness is for thinner continent outline than default of 1.5
cfp.levs(min=0,max=100,step=10)
cfp.cscale('WhiteBlue')
cfp.con(my_sws_mean, lines=False, blockfill=True, colorbar_title='') # We have made sure that both contour lines and shading are switched off to have just outline map of UK
for i in range(len(st_name)):
    cfp.plotvars.mymap.plot(float(st_lon[i]),float(st_lat[i]), linewidth=3.0, marker='o', color='red', transform=ccrs.PlateCarree())
    cfp.plotvars.mymap.text(float(st_lon[i])-0.2,float(st_lat[i])+0.05, st_id[i], color='red',fontsize=8, horizontalalignment='center', transform=ccrs.PlateCarree())

cfp.gclose()

# cfp.con refers to contour plots, for more types of plots please see http://ajheaps.github.io/cf-plot/gallery.html
# my_dataset[0,:,:] uses subspacing by index, for more detail please see https://ncas-cms.github.io/cf-python/tutorial.html?#subspacing-by-index

<a id="manipulating_data"></a>
### Manipulating the datasets

In [ ]:
# We learn how to select specific station data for the whole 366 days of the leap year as an numpy array
# For example, we would like to compare the two northermost stations, "ALIC1", against the nearest station "CHOBH"
# To find the station data from just the name, we need to find the index for the station in the first row of my_vwc
print(my_vwc[0,:]) # First row of my_vwc
# Please note that each station id has "_VWC" at the end, so we need to add that in the query to find the index for the respective station
#For example, to find station "ALIC1" index, we need to search for "ALIC1_VWC"

In [ ]:
# Command to find the index for station ALIC1 and CHOBH from the station data
alic1_idx = np.where(my_vwc[0,:]=='ALIC1_VWC')[0][0]
chobh_idx = np.where(my_vwc[0,:]=='CHOBH_VWC')[0][0]
print('Index for ALIC1 station is '+str(alic1_idx))
print('Index for CHOBH station is '+str(chobh_idx))


In [ ]:
# Using the indices for the stations fromt the cell above we can extract data for both of the stations
alic1_vwc = my_vwc[1:,alic1_idx].tolist() # We use 1: to remove the station name from the list and tolist to convert to list
print(alic1_vwc) # All the data is in string format, which we will convert to float and use NaN values for empty strings
alic1_vwc = np.array([np.nan if x == '' else float(x) for x in alic1_vwc]) # We use list conversion to change the strings to float
print(alic1_vwc)
print(alic1_vwc.shape)

# We do the same for CHOBH station VWC data
chobh_vwc = my_vwc[1:,chobh_idx].tolist()
chobh_vwc = np.array([np.nan if x == '' else float(x) for x in chobh_vwc])


In [ ]:
# Identifying the model grid point nearest to the ALIC1 and CHOBH stations
# First step is to find the latitude and longitude for the two stations
alic1_idx = np.where(my_stations=='ALIC1')[0][0]
chobh_idx = np.where(my_stations=='CHOBH')[0][0]

print(my_stations[alic1_idx,:])
print(my_stations[chobh_idx,:])


In [ ]:
# Printing out the latitude longtiude information for the modelled data
print(my_sws.coord('X').array)
print(my_sws.coord('Y').array)


In [ ]:
# Using the latitude longitude information from the stations and the modelled grid above, we try to find the nearest grid point in the modelled data
# For both station the nearest grid point on the modelled grid is X = -0.75; Y = 51.25
# Subselect these two stations from the modelled netcdf file

model_sws = my_sws.subspace(X = -0.75, Y = 51.25).squeeze().array #We convert it to a numpy array
model_sws


In [ ]:
# Identify the time information from the first column of my_vwc
time = my_vwc[1:,0]
print(time)


In [ ]:
# Plotting the two stations VWC data for the year 2016 as line plot, also called a spaghetti plot
fig = plt.figure(figsize=(18,7))
ax=plt.subplot(1,1,1)
plt.plot(time, alic1_vwc, color='blue', label='ALIC1_VWC')
plt.plot(time, chobh_vwc, color='red', label='CHOBH_VWC')
plt.plot(time, model_sws, color='k', label='Model_SWS')
plt.ylabel('Soil Water (Percentage)')
plt.xticks(time[0::30])
plt.xlabel('Time (DD/MM/YYYY)')
plt.legend()
plt.show()
#For more information about using matplotlib please see https://matplotlib.org/


**Discussion**: The SWS and VWC separately show similar variability for 2016, which can be attributed to similar meteorology and geogrpahy of the region, but VWC from the COSMOS stations are still not directly comparable to the SWS data from the model. Both of these variables are indicators for percentage of moisture/water within the soil, but there are some differences between station data and model data, which can mostly be attributed to differences in the variables compared, but can also be attributed to model uncertainity and coarse resolution of the modelled data among many other. In this demonstration we are providing you with tools and methods to compare modelled and observed data (avaialble in different formats) using DataLabs platform. 

<a id="#further_exercises"></a>
### Further Exercises

If you are finished with the rest of the Notebook above, please try the following excercises in the cells given below the exercise.

1. Please plot line graphs for 2016 for BUNNY and STGHT stations and compare them to the modelled grid point data that they are nearest to.
   
   HINT: Follow the method demonstrated in the Notebook.

2. Please plot line graphs for daily minimum, maximum and mean VWC for all the COSMOS stations across UK and compare to the minimum, maximum and mean SWS from modelled data over the whole UK for all the days of 2016. Does this comparison seem correct? Is there any area that may not be represented by the stations? What can be done to correct this?
   
   HINT: You will have to apply numpy minimum, maximum and mean functions across all the rows of VWC data so that you end up with 366 days of output for each function. For the SWS model data, please use collapse function for the whole area. 

3. Please plot the average VWC for all stations south and east of BUNNY (including BUNNY) and compare it to all the grid points south and east of BUNNY (including the grid point that BUNNY falls on). 
   
   HINT: You can create a python loop to extract data from all the stations that you need. As demonstrated in the cf-python examples Notebook you can use the cf.wi() function to select the range of the X and Y coordinates (south and east of BUNNY) and then take an area mean of the subspaced region. 